In [1]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Dropout, BatchNormalization, GaussianNoise
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [10]:
size = 256
channels = 1  #input image format
input_img = Input(shape=(size, size, channels))
#print(input_img)

In [16]:
#Encoder
x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# Decoder
x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)

In [17]:
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
autoencoder.summary()

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 256, 256, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 256, 256, 16)   │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 128, 128, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 128, 128, 8)    │         1,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 64, 64, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 64, 64, 8)      │           584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 32, 32, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 32, 32, 8)      │           584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_6 (UpSampling2D)  │ (None, 64, 64, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 64, 64, 8)      │           584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_7 (UpSampling2D)  │ (None, 128, 128, 8)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 128, 128, 16)   │         1,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_8 (UpSampling2D)  │ (None, 256, 256, 16)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 256, 256, 1)    │           145 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,385 (17.13 KB)

 Trainable params: 4,385 (17.13 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
def preprocess_image(image):
    #resize image so they're same format
    image = tf.image.resize(image, (size, size), method='lanczos5')
    return image


datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=preprocess_image,
    validation_split=0.15
)


In [6]:
import os
os.getcwd()

'/Users/yong/Desktop/thesis'

In [19]:
train_generator = datagen.flow_from_directory(
    'dataset',  #path to dataset
    target_size=(size, size),
    color_mode='grayscale',
    batch_size=32,
    class_mode='input',
    subset='training'
)
validation_generator = datagen.flow_from_directory(
    'dataset',  #path to dataset
    target_size=(size, size),
    color_mode='grayscale',
    batch_size=32,
    class_mode='input',
    subset='validation'
)

Found 152 images belonging to 6 classes.


Found 25 images belonging to 6 classes.


In [20]:
# Define early stopping callback
early_stopper = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)

# Now include the callback in the fit method
autoencoder.fit(
    train_generator,
    epochs=40,
    validation_data=validation_generator,
    callbacks=[early_stopper],
    shuffle = True
)

Epoch 1/40


/Users/yong/anaconda3/envs/plard/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 525ms/step - loss: 0.2446 - val_loss: 0.2394
Epoch 2/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 424ms/step - loss: 0.2386 - val_loss: 0.2313
Epoch 3/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 399ms/step - loss: 0.2289 - val_loss: 0.2110
Epoch 4/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 393ms/step - loss: 0.2037 - val_loss: 0.1633
Epoch 5/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 378ms/step - loss: 0.1472 - val_loss: 0.0765
Epoch 6/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 395ms/step - loss: 0.0575 - val_loss: 0.0143
Epoch 7/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 375ms/step - loss: 0.0111 - val_loss: 0.0045
Epoch 8/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 375ms/step - loss: 0.0038 - val_loss: 0.0035
Epoch 9/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 400ms/step - loss: 0.0030 - val_loss: 0.0031
Epoch 10/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 377ms/step - loss: 0.0026 - val_loss: 0.0031
Epoch 11/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 382ms/step - loss: 0.0025 - val_loss: 0.0029
Epoch 12/40
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 377ms/step - loss: 0.0023 - val_loss: 0.0028


In [21]:
#save model
autoencoder.save('0505x2.keras')